# XGBoost with Covariate Shift Correction
Extension of Model 0v2 that has covariate shift via Kullback-Leibler Importance Estimation Procedure integrated.

In [1]:
# Load libraries
import numpy as np
import pandas as pd
import xgboost as xgb

import pickle
import pdb
import os

from scipy.stats import skew, kurtosis

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin  # For making custom classes
from sklearn.externals.joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.model_selection import KFold

from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion

In [2]:
# High-level parameters
debug=True
random_state=0

## Helper Functions and Classes

In [3]:
class UniqueTransformer(BaseEstimator, TransformerMixin):
    '''
    Class with fit and transform methods for removing duplicate columns from a dataset
    **fit** finds the indexes of unique columns using numpy unique
    **transform** returns the dataset with the indexes of unique columns
    '''
    def __init__(self, axis=1):
        self.axis=axis

    def fit(self, X, y=None):
        print 'Finding unique indexes...'
        _, self.unique_indexes_ = np.unique(X, axis=self.axis, return_index=True)
        return self

    def transform(self, X, y=None):
        print 'Filtering for only unique columns...'
        return X[:, self.unique_indexes_]

In [4]:
class ClassifierTransformer(BaseEstimator, TransformerMixin):
    '''
    Class describing an object that transforms datasets via estimator results
    **_get_labels** specifies target value bins and transforms target vector into bin values
    '''
    def __init__(self, estimator=None, n_classes=2, cv=3):
        self.estimator=estimator
        self.n_classes=n_classes
        self.cv=cv

    def _get_labels(self, y):
        y_labels = np.zeros(len(y))
        y_us = np.sort(np.unique(y))
        step = int(len(y_us)/self.n_classes)

        for i_class in range(self.n_classes):
            if i_class+1 == self.n_classes:  # Edge case where i_class is initialized at 1
                y_labels[y >= y_us[i_class*step]] = i_class
            else:
                y_labels[np.logical_and(y>=y_us[i_class*step], y<y_us[(i_class+1)*step])] = i_class
        return y_labels

    def fit(self, X, y):
        print 'Fitting random forest classifier with n_classes = %s'%self.n_classes
        y_labels = self._get_labels(y)
        kf = KFold(n_splits=self.cv, shuffle=False, random_state=random_state)
        self.estimators_ = []
        # Train individual classifiers
        for train, _ in kf.split(X, y_labels):
            self.estimators_.append(clone(self.estimator).fit(X[train], y_labels[train]))
        return self

    def transform(self, X, y=None):
        print 'Applying classifier transformation with n_classes = %s'%self.n_classes
        kf = KFold(n_splits=self.cv, shuffle=False, random_state=random_state)

        X_prob = np.zeros((X.shape[0], self.n_classes))
        X_pred = np.zeros(X.shape[0])

        for estimator, (_, test) in zip(self.estimators_, kf.split(X)):
            X_prob[test] = estimator.predict_proba(X[test])
            X_pred[test] = estimator.predict(X[test])
        return np.hstack([X_prob, np.array([X_pred]).T])

In [5]:
# Function for transforming a row into statistical values
def apply_stats_to_row(row):
    stats = []
    for fun in stat_functions:
        stats.append(fun(row))
    return stats

In [6]:
class StatsTransformer(BaseEstimator, TransformerMixin):
    '''
    Class describing an object for transforming datasets into statistical values row-wise
    NOTE: This class is dependent on the function **apply_stats_to_row**
    '''
    def __init__(self, verbose=0, n_jobs=-1, pre_dispatch='2*n_jobs'):
        self.verbose = verbose
        self.n_jobs = n_jobs
        self.pre_dispatch = pre_dispatch

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        print 'Applying statistical transformation to dataset...'
        parallel = Parallel(n_jobs=self.n_jobs, pre_dispatch=self.pre_dispatch, verbose=self.verbose)
        # Get statistics transformation
        stats_list = parallel(delayed(apply_stats_to_row)(X[i_smpl, :]) for i_smpl in range(len(X)))
        return np.array(stats_list)

In [7]:
class _StatFunAdaptor:
    '''
    Class describing an object that wraps pre-processing functions with a main statistical function
    **__init__** sets up the object parameters
    **__call__** describes routine steps when object is called
    '''
    def __init__(self, stat_fun, *funs, **stat_fun_kwargs):
        self.stat_fun = stat_fun
        self.funs = funs
        self.stat_fun_kwargs = stat_fun_kwargs

    def __call__(self, x):
        x = x[x != 0]  # Only look at nonzero entries
        # Transform row with cached functions
        for fun in self.funs:
            x = fun(x)
        if x.size == 0:
            return -99999  # Edge case default
        return self.stat_fun(x, **self.stat_fun_kwargs)  # Returns result of a run

In [8]:
def diff2(x):
    return np.diff(x, n=2)

In [9]:
def get_stat_funs():
    '''
    Function for defining all the statistical functions used for evaluating elements in a row-wise manner
    Functions include: length, minimum, maximum, standard deviation, skew, kurtosis, and percentile
    '''
    stat_funs = []

    stats = [len, np.min, np.max, np.median, np.std, skew, kurtosis] + 19 * [np.percentile]
    # Dictionary arguments (nontrivial only for percentile function)
    stats_kwargs = [{} for i in range(7)] + [{'q': i} for i in np.linspace(0.05, 0.95, 19)]

    for stat, stat_kwargs in zip(stats, stats_kwargs):
        stat_funs.append(_StatFunAdaptor(stat, **stat_kwargs))
        stat_funs.append(_StatFunAdaptor(stat, np.diff, **stat_kwargs))  # Apply to 1-diff of row
        stat_funs.append(_StatFunAdaptor(stat, diff2, **stat_kwargs))  # Apply to 2-diff of row
        stat_funs.append(_StatFunAdaptor(stat, np.unique, **stat_kwargs))  # Apply to unique vals of row
        stat_funs.append(_StatFunAdaptor(stat, np.unique, np.diff, **stat_kwargs))  # Apply to unique, 1-diff row vals
        stat_funs.append(_StatFunAdaptor(stat, np.unique, diff2, **stat_kwargs))  # Apply to unique, 2-diff row vals
    return stat_funs

In [10]:
# Function for retrieving a Random Forest Classifier object
def get_rfc():
    return RandomForestClassifier(n_estimators=100,
                                  max_features=0.5,
                                  max_depth=None,
                                  max_leaf_nodes=270,
                                  min_impurity_decrease=0.0001,
                                  random_state=123,
                                  n_jobs=-1)

In [11]:
# Function for setting up datasets
def get_input(debug=False):
    if debug:
        print 'Loading debug train and test datasets...'
        train = pd.read_csv('../data/train_debug.csv')
        test = pd.read_csv('../data/test_debug.csv')
    else:
        print 'Loading original train and test datasets...'
        train = pd.read_csv('../data/train.csv')
        test = pd.read_csv('../data/test.csv')
    y_train_log = np.log1p(train['target'])
    id_test = test['ID']
    # Drop unnecessary columns
    train.drop(labels=['ID', 'target'], axis=1, inplace=True)
    test.drop(labels=['ID'], axis=1, inplace=True)
    # Find shape of loaded datasets
    print('Shape of training dataset: {} Rows, {} Columns'.format(*train.shape))
    print('Shape of test dataset: {} Rows, {} Columns'.format(*test.shape))

    return train.values, y_train_log.values, test.values, id_test.values

In [12]:
# Function for retrieving width list
def get_width(debug=False):
    '''
    Function for loading either debug or full width lists
    '''
    if debug:
        return [10, 1000]
    else:
        return [0.1, 1, 10, 100, 1000, 10000, 100000]

In [13]:
# Function for calculating Gaussian kernel value
def calc_gaussian(x, center, width):
    return np.exp(-(np.square(np.linalg.norm(np.subtract(x, center))))/(2*np.square(width)))

In [14]:
# Function for calculating importance weights
def get_importances(data, alpha, kc, kw):
    importance_weights = np.zeros(len(data))
    for i, row in enumerate(data):
        kernel_sum = 0
        for j, center in enumerate(kc):
            kernel_sum += alpha[j]*calc_gaussian(row, center, kw)
        importance_weights[i] = kernel_sum
    return importance_weights

In [15]:
# Kullback-Leibler Importance Estimation Procedure training function
def train_KLIEP(train, test, num_kernels=100, kernel_width=10, lr=0.001, a_val=1, stop=0.00001):
    '''
    Function for getting KLIEP weights for a given training and test set
    '''
    # Instantiate kernel centers
    kernel_idx_bag = np.random.permutation(len(test))
    kernel_idx = np.array([np.random.choice(kernel_idx_bag) for i in range(num_kernels)])
    kernel_centers = test[kernel_idx, :]
    # Compute A matrix
    A = np.zeros(shape=(len(test), len(kernel_centers)))
    for i, row in enumerate(test):
        for j, center in enumerate(kernel_centers):
            A[i, j] = calc_gaussian(row, center, kernel_width)
    # Compute b vector
    b = np.zeros(num_kernels)
    for j, center in enumerate(kernel_centers):
        temp_sum = 0
        for row in train:
            temp_sum += calc_gaussian(row, center, kernel_width)
        b[j] = temp_sum/np.float16(len(train))
    # Initialize alpha vector
    alpha = a_val * np.ones(shape=num_kernels)
    # Begin training
    alpha_old = np.zeros(shape=num_kernels)
    counter = 0
    while True:
        alpha = np.add(alpha, lr*np.matmul(A.T, np.divide(np.ones(len(test)), np.matmul(A, alpha))))
        alpha = np.add(alpha, np.divide(np.multiply((1-np.dot(b, alpha)), b), np.dot(b, b)))
        alpha = np.maximum(np.zeros(num_kernels), alpha)
        alpha = np.divide(alpha, np.dot(b, alpha))
        # Check convergence by average deviation
        deviation = np.linalg.norm(np.subtract(alpha, alpha_old))
        if deviation < stop*np.linalg.norm(alpha_old):
            print 'Converged in %s iterations!'%counter
            importance_weights = get_importances(data=test, alpha=alpha, kc=kernel_centers, kw=kernel_width)
            return importance_weights, alpha, kernel_centers
            break
        else:
            counter += 1
            alpha_old = alpha

In [16]:
# Function for getting the best model
def get_best_KLIEP(train, test, width_list, n_splits=10, num_kernels=100, lr=0.001, a_val=1, stop=0.00001):
    '''
    Function for tuning KLIEP kernel performance
    '''
    # Split test set into disjoint subsets
    split_sets = []
    kf = KFold(n_splits=n_splits, shuffle=False, random_state=random_state)
    print 'Splitting test set into %s disjoint subsets...'%n_splits
    for _, test_idx in kf.split(test):
        split_sets.append(test[test_idx, :])
    # Evaluate each model
    j_models = []
    alpha_list = []
    centers_list = []
    for idx, w in enumerate(width_list):
        print 'Working on split set %s'%idx
        print 'Evaluating KLIEP model with Gaussian kernel width of %s...'%w
        j_avglist = []
        for s in split_sets:
            importance, alpha, center = train_KLIEP(train=train, test=s, num_kernels=num_kernels, kernel_width=w, 
                                     lr=lr, a_val=a_val, stop=stop)
            j_avglist.append(np.mean(np.log(importance)))
        j_models.append(np.mean(j_avglist))
        alpha_list.append(alpha)
        centers_list.append(center)
    # Use best model to evaluate train set KLIEP importances
    best_idx = np.argmax(np.array(j_models))
    print '\nBest width was: %s'%width_list[best_idx]
    importance_weights = get_importances(data=train, alpha=alpha_list[best_idx], kc=centers_list[best_idx], 
                                         kw=width_list[best_idx])
    return width_list[best_idx], importance_weights

## Main Script

In [17]:
# Get data
X_train, y_train_log, X_test, id_test = get_input(debug)
# Load width list
wlist = get_width(debug)

Loading debug train and test datasets...
Shape of training dataset: 100 Rows, 4992 Columns
Shape of test dataset: 200 Rows, 4992 Columns


In [18]:
# Remove constant columns
variance_checker = VarianceThreshold(threshold=0.0)
xtrain = variance_checker.fit_transform(X_train)
xtest = variance_checker.transform(X_test)

# Remove duplicate columns
unique_transformer = UniqueTransformer()
unique_transformer.fit(X_train)
xtrain = unique_transformer.transform(X_train)
xtest = unique_transformer.transform(X_test)

Finding unique indexes...
Filtering for only unique columns...
Filtering for only unique columns...


In [19]:
# Define stat functions
stat_functions = get_stat_funs()

In [20]:
# Define feature union
data_union = FeatureUnion([
    ('pca', PCA(n_components=100)),
    ('ct-2', ClassifierTransformer(get_rfc(), n_classes=2, cv=5)),
    ('ct-3', ClassifierTransformer(get_rfc(), n_classes=3, cv=5)),
    ('ct-4', ClassifierTransformer(get_rfc(), n_classes=4, cv=5)),
    ('ct-5', ClassifierTransformer(get_rfc(), n_classes=5, cv=5)),
    ('st', StatsTransformer(verbose=2))
])

In [21]:
data_union.fit(X=xtrain, y=y_train_log)
print 'Creating processed training set...'
train_data = data_union.transform(xtrain)
print 'Creating processed test set...'
test_data = data_union.transform(xtest)

Fitting random forest classifier with n_classes = 2
Fitting random forest classifier with n_classes = 3
Fitting random forest classifier with n_classes = 4
Fitting random forest classifier with n_classes = 5
Creating processed training set...
Applying classifier transformation with n_classes = 2
Applying classifier transformation with n_classes = 3
Applying classifier transformation with n_classes = 4
Applying classifier transformation with n_classes = 5
Applying statistical transformation to dataset...


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished


Creating processed test set...
Applying classifier transformation with n_classes = 2
Applying classifier transformation with n_classes = 3
Applying classifier transformation with n_classes = 4
Applying classifier transformation with n_classes = 5
Applying statistical transformation to dataset...


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    0.8s finished


In [22]:
# Scale data
xdata = np.concatenate([train_data, test_data], axis=0)
scaler = StandardScaler()
xdata_scaled = scaler.fit_transform(X=xdata)
train_scaled = xdata_scaled[:len(X_train), :]
test_scaled = xdata_scaled[len(X_train):, :]

In [23]:
# # Get KLIEP importance weights
# weights_path = './covariate_shift/kliep_weights.pickle'
# if os.path.exists(weights_path):
#     print 'Loading existing KLIEP importances...'
#     with open(weights_path, 'rb') as handle:
#         best_width, importances = pickle.load(handle)
# else:
#     print 'Generating new KLIEP importances...'
#     best_width, importances = get_best_KLIEP(train_data, test_data, wlist)

In [24]:
# Train XGBoost Regressor
# train_KLIEP(train_scaled, test_scaled)